<a href="https://colab.research.google.com/github/mehrnazeraeisi/Best-Features-Group-Results-on-KNN/blob/main/Best_Features_Group_Results_on_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def knn_auto_evaluator_from_xy(X_data, y_data, test_sizes=[0.1, 0.13, 0.16 , 0.19, 0.25], n_repeats=20):
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
    from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
    import warnings
    warnings.filterwarnings("ignore")

    # تشخیص نوع تارگت
    if pd.api.types.is_numeric_dtype(y_data):
        target_type = 'regression' if len(np.unique(y_data)) > 10 else 'classification'
    else:
        target_type = 'classification'

    feature_names = X_data.columns.tolist()
    results = []

    for ts in test_sizes:
        for repeat in range(1, n_repeats+1):
            X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=ts)

            scores = []
            for feature in feature_names:
                model = KNeighborsRegressor() if target_type == 'regression' else KNeighborsClassifier()
                model.fit(X_train[[feature]], y_train)
                y_pred = model.predict(X_train[[feature]])
                score = r2_score(y_train, y_pred) if target_type == 'regression' else accuracy_score(y_train, y_pred)
                scores.append((feature, score))

            df_scores = pd.DataFrame(scores, columns=['Feature', 'Score']).sort_values(by='Score', ascending=False)

            best_score = -np.inf
            best_group = []
            for k in range(1, len(df_scores)+1):
                group = df_scores['Feature'][:k].tolist()
                model = KNeighborsRegressor() if target_type == 'regression' else KNeighborsClassifier()
                model.fit(X_train[group], y_train)
                y_pred = model.predict(X_train[group])
                score = r2_score(y_train, y_pred) if target_type == 'regression' else accuracy_score(y_train, y_pred)
                if score > best_score:
                    best_score = score
                    best_group = group

            model_all = KNeighborsRegressor() if target_type == 'regression' else KNeighborsClassifier()
            model_best = KNeighborsRegressor() if target_type == 'regression' else KNeighborsClassifier()
            model_all.fit(X_train, y_train)
            model_best.fit(X_train[best_group], y_train)
            y_pred_all = model_all.predict(X_test)
            y_pred_best = model_best.predict(X_test[best_group])

            if target_type == 'regression':
                result_row = {
                    'Test Size': ts,
                    'Repeat': repeat,
                    'All_R2': r2_score(y_test, y_pred_all),
                    'All_MSE': mean_squared_error(y_test, y_pred_all),
                    'BestGroup_R2': r2_score(y_test, y_pred_best),
                    'BestGroup_MSE': mean_squared_error(y_test, y_pred_best),
                    'BestGroup Size': len(best_group)
                }
                column_order = ['Test Size', 'Repeat', 'All_R2', 'All_MSE', 'BestGroup_R2', 'BestGroup_MSE', 'BestGroup Size']
            else:
                result_row = {
                    'Test Size': ts,
                    'Repeat': repeat,
                    'All_Accuracy': accuracy_score(y_test, y_pred_all),
                    'BestGroup_Accuracy': accuracy_score(y_test, y_pred_best),
                    'BestGroup Size': len(best_group)
                }
                column_order = ['Test Size', 'Repeat', 'All_Accuracy', 'BestGroup_Accuracy', 'BestGroup Size']

            results.append(result_row)

    # ساخت DataFrame نهایی با ترتیب دقیق ستون‌ها
    return pd.DataFrame(results)[column_order]


In [ ]:
!pip install openml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.8 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=899adbd6ff59ef362097b34bb0817c46f55001f663eee41aa0a9b0d2e009a12b
  Stored in directory: /root/.cache/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


In [ ]:
results = []

In [ ]:
import pandas as pd

# بارگذاری فایل
data = pd.read_csv('bodyfat.csv')


# جدا کردن ویژگی‌ها و برچسب‌ها
X = data.drop('Pct.BF', axis=1)
y= data['Pct.BF']


print(X.shape)


df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Bodyfat', 'type': 'regression', 'table': df_results})

(250, 14)
    Test Size  Repeat    All_R2    All_MSE  BestGroup_R2  BestGroup_MSE  \
0        0.10       1  0.555352  23.137072      0.670260      17.157888   
1        0.10       2  0.536302  46.193088      0.619361      37.918896   
2        0.10       3  0.490499  24.796928      0.570165      20.919664   
3        0.10       4  0.527760  45.250720      0.501917      47.727088   
4        0.10       5  0.275914  38.084192      0.435546      29.688160   
..        ...     ...       ...        ...           ...            ...   
95       0.25      16  0.586683  27.870819      0.646345      23.847651   
96       0.25      17  0.389786  34.570083      0.531327      26.551441   
97       0.25      18  0.524661  35.620508      0.607891      29.383530   
98       0.25      19  0.677172  25.357016      0.682847      24.911244   
99       0.25      20  0.533545  29.795054      0.554464      28.458851   

    BestGroup Size  
0                7  
1                6  
2                8  
3    

In [ ]:
import pandas as pd

# بارگذاری فایل
data = pd.read_csv('advertising.csv')



# جدا کردن ویژگی‌ها و برچسب‌ها
X = data.drop('sales', axis=1)
y= data['sales']

print(X.shape)

df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Advertising', 'type': 'regression', 'table': df_results})

(200, 3)
    Test Size  Repeat    All_R2   All_MSE  BestGroup_R2  BestGroup_MSE  \
0        0.10       1  0.914370  2.634060      0.957630       1.303340   
1        0.10       2  0.930709  2.666880      0.937787       2.394460   
2        0.10       3  0.926909  1.908440      0.956694       1.130740   
3        0.10       4  0.884607  2.480520      0.952983       1.010680   
4        0.10       5  0.844926  3.043920      0.946899       1.042320   
..        ...     ...       ...       ...           ...            ...   
95       0.25      16  0.898815  2.737536      0.934784       1.764392   
96       0.25      17  0.917961  2.512840      0.962533       1.147608   
97       0.25      18  0.842689  4.116392      0.914633       2.233808   
98       0.25      19  0.946872  1.421024      0.966895       0.885464   
99       0.25      20  0.925384  2.280056      0.971703       0.864680   

    BestGroup Size  
0                2  
1                2  
2                2  
3                2

In [ ]:
import pandas as pd

# بارگذاری فایل
data = pd.read_csv('BostonHousing.csv')



# جدا کردن ویژگی‌ها و برچسب‌ها
X = data.drop('medv', axis=1)
y= data['medv']

print(X.shape)

df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)
results.append({'name': 'BostonHousing', 'type': 'regression', 'table': df_results})

(506, 13)
    Test Size  Repeat    All_R2    All_MSE  BestGroup_R2  BestGroup_MSE  \
0        0.10       1  0.554713  40.346408      0.658481      30.944220   
1        0.10       2  0.471320  31.458329      0.784518      12.821961   
2        0.10       3  0.599887  37.111780      0.773419      21.016157   
3        0.10       4  0.416010  52.796204      0.874818      11.317208   
4        0.10       5  0.664370  32.808400      0.870985      12.611412   
..        ...     ...       ...        ...           ...            ...   
95       0.25      16  0.422403  43.199291      0.827206      12.923546   
96       0.25      17  0.575663  44.787874      0.724967      29.029162   
97       0.25      18  0.477836  47.456302      0.664525      30.489276   
98       0.25      19  0.407476  60.848082      0.748888      25.787455   
99       0.25      20  0.474259  41.126293      0.794722      16.057950   

    BestGroup Size  
0                4  
1                5  
2                4  
3    

In [ ]:
import pandas as pd

# بارگذاری فایل
data = pd.read_csv('cookie_recipes.csv')


# جدا کردن ویژگی‌ها و برچسب‌ها
X = data.drop('rating', axis=1)
y= data['rating']

print(X.shape)

df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Cookie_Recipes', 'type': 'regression', 'table': df_results})

(50, 5)
    Test Size  Repeat    All_R2   All_MSE  BestGroup_R2  BestGroup_MSE  \
0        0.10       1 -0.392690  2.381500      0.959357       0.069500   
1        0.10       2  0.117323  0.560500      0.925984       0.047000   
2        0.10       3 -1.144186  0.461000      0.846512       0.033000   
3        0.10       4 -0.076582  0.850500      0.996835       0.002500   
4        0.10       5  0.190000  0.243000      0.831667       0.050500   
..        ...     ...       ...       ...           ...            ...   
95       0.25      16 -0.055835  1.436154      0.853812       0.198846   
96       0.25      17 -0.137970  2.271731      0.945572       0.108654   
97       0.25      18  0.179133  0.714615      0.915395       0.073654   
98       0.25      19 -0.238179  2.172308      0.932369       0.118654   
99       0.25      20  0.047668  0.975577      0.906137       0.096154   

    BestGroup Size  
0                2  
1                2  
2                2  
3                2 

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd


# بارگذاری داده‌های قیمت خانه‌های کالیفرنیا
data =  fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y= data.target

print(X.shape)

X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'California_Housing', 'type': 'regression', 'table': df_results})

(20640, 8)
    Test Size  Repeat    All_R2   All_MSE  BestGroup_R2  BestGroup_MSE  \
0        0.10       1  0.163020  1.095677      0.743952       0.335188   
1        0.10       2  0.171633  1.028354      0.757905       0.300542   
2        0.10       3  0.135847  1.126090      0.778945       0.288060   
3        0.10       4  0.167912  1.122969      0.762963       0.319900   
4        0.10       5  0.149857  1.120573      0.749294       0.330455   
..        ...     ...       ...       ...           ...            ...   
95       0.25      16  0.116332  1.155728      0.758040       0.316454   
96       0.25      17  0.168299  1.111167      0.762850       0.316837   
97       0.25      18  0.141340  1.122581      0.752954       0.322978   
98       0.25      19  0.151238  1.125774      0.742814       0.341124   
99       0.25      20  0.150537  1.157078      0.752526       0.337092   

    BestGroup Size  
0                6  
1                6  
2                6  
3               

In [ ]:
from sklearn.datasets import load_wine
data = load_wine(as_frame=True)
X = data.data
y = data.target

df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'wine', 'type': 'classification', 'table': df_results})

    Test Size  Repeat  All_Accuracy  BestGroup_Accuracy  BestGroup Size
0        0.10       1      0.777778            1.000000               2
1        0.10       2      0.666667            0.888889               3
2        0.10       3      0.888889            0.833333               1
3        0.10       4      0.666667            0.888889               3
4        0.10       5      0.500000            0.722222               1
..        ...     ...           ...                 ...             ...
95       0.25      16      0.711111            0.955556               2
96       0.25      17      0.733333            0.933333               5
97       0.25      18      0.644444            0.844444               2
98       0.25      19      0.666667            0.733333               1
99       0.25      20      0.666667            0.800000               2

[100 rows x 5 columns]


In [ ]:
import openml
import pandas as pd

# دانلود دیتاست
dataset = openml.datasets.get_dataset(43977)
data, y_data, _, _ = dataset.get_data(target=dataset.default_target_attribute)

# افزودن ستون هدف به دیتافریم کامل
data['rating'] = y_data  # فرض بر این است که ستون هدف، rating است

# جدا کردن ویژگی‌ها و برچسب‌ها
X = data.drop('rating', axis=1)
y = data['rating']

print(X.shape)

df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Jannis(43977)', 'type': 'classification', 'table': df_results})

(57580, 54)


In [ ]:
import pandas as pd

# بارگذاری فایل اکسل
data = pd.read_excel('Pistachio.xlsx')
# حذف سطرهای حاوی NaN
data = data.dropna()


# جدا کردن ویژگی‌ها و برچسب‌ها
X = data.drop('Class', axis=1)
y = data['Class']

print(data.shape)


df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Pistachio', 'type': 'classification', 'table': df_results})

(2148, 17)
    Test Size  Repeat  All_Accuracy  BestGroup_Accuracy  BestGroup Size
0        0.10       1      0.790698            0.823256               2
1        0.10       2      0.800000            0.865116               5
2        0.10       3      0.725581            0.837209               4
3        0.10       4      0.706977            0.837209               6
4        0.10       5      0.800000            0.888372               7
..        ...     ...           ...                 ...             ...
95       0.25      16      0.739292            0.841713               5
96       0.25      17      0.756052            0.826816               3
97       0.25      18      0.728119            0.811918               2
98       0.25      19      0.729981            0.821229               4
99       0.25      20      0.765363            0.834264               4

[100 rows x 5 columns]


In [ ]:
#
import openml
import pandas as pd

# دانلود دیتاست
dataset = openml.datasets.get_dataset(43971)
data, y_data, _, _ = dataset.get_data(target=dataset.default_target_attribute)
print(data.shape)
# افزودن ستون هدف به دیتافریم کامل
data['rating'] = y_data  # فرض بر این است که ستون هدف، rating است


# جدا کردن ویژگی‌ها و برچسب‌ها
X= data.drop('rating', axis=1)
y= data['rating']

print(X.shape)


df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'MagicTelescope(43971)', 'type': 'classification', 'table': df_results})

(13376, 10)
قبل از حذف NaN و ستون: (13376, 11)
Index(['fLength:', 'fWidth:', 'fSize:', 'fConc:', 'fConc1:', 'fAsym:',
       'fM3Long:', 'fM3Trans:', 'fAlpha:', 'fDist:', 'rating'],
      dtype='object')
   fLength:  fWidth:  fSize:  fConc:  fConc1:   fAsym:  fM3Long:  fM3Trans:  \
0   69.2979  26.8809  3.1930  0.2065   0.1074  39.6296   44.3457   -23.0604   
1   24.5939  10.1418  2.5676  0.5007   0.2693  -8.4503   15.2452    -7.0283   
2   55.4800  27.1606  3.1826  0.2299   0.1225  43.1016   54.2556    13.7406   
3   12.6594  11.7413  2.1351  0.7033   0.3846 -15.8596    9.4522    -8.7126   
4   38.6204  20.5632  2.9770  0.2478   0.1270 -13.8229  -31.3983   -13.1337   

   fAlpha:   fDist: rating  
0   9.3234  248.750      g  
1  17.0056  173.288      g  
2  32.2220  262.181      g  
3  43.5434  227.711      g  
4   5.8671  192.467      g  
بعد از حذف: (13376, 11)
(13376, 10)
    Test Size  Repeat  All_Accuracy  BestGroup_Accuracy  BestGroup Size
0        0.10       1      0.765321    

In [ ]:

import openml
import pandas as pd

dataset = openml.datasets.get_dataset(1046)
data, y_data, _, _ = dataset.get_data(target=dataset.default_target_attribute)

# 🔧 افزودن target به dataframe کامل (مثل خواندن فایل CSV)
data['rating'] = y_data  # فرض می‌گیریم نام ستون هدف "rating" باشه برای یکسان بودن با مثال شما

print(data.shape)
# حذف سطرهای حاوی NaN
data = data.dropna()

print(data.shape)

# 🎯 جدا کردن ویژگی‌ها و برچسب‌ها (مطابق ساختار شما)
X = data.drop('rating', axis=1)
y = data['rating']
data_features = X.columns


df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Hill_Valley_with_Noise(1046)', 'type': 'classification', 'table': df_results})

In [ ]:
# Climate Model Simulation
# شبیه‌سازی آب‌وهوا (دوتایی)
import openml
import pandas as pd

dataset = openml.datasets.get_dataset(40966)
data, y_data, _, _ = dataset.get_data(target=dataset.default_target_attribute)

# 🔧 افزودن target به dataframe کامل (مثل خواندن فایل CSV)
data['rating'] = y_data  # فرض می‌گیریم نام ستون هدف "rating" باشه برای یکسان بودن با مثال شما

print(data.shape)
# حذف سطرهای حاوی NaN
data = data.dropna()


print(data.shape)

# 🎯 جدا کردن ویژگی‌ها و برچسب‌ها (مطابق ساختار شما)
X = data.drop('rating', axis=1)
y = data['rating']


df_results = knn_auto_evaluator_from_xy(X, y)
print(df_results)

results.append({'name': 'Climate Model Simulation', 'type': 'classification', 'table': df_results})

In [ ]:
import pandas as pd

def summarize_max_diff_results(dataset_tables):
    """
    dataset_tables: لیستی از دیکشنری‌ها به شکل زیر:
    [
        {'name': 'Diabetes', 'type': 'regression', 'table': df1},
        {'name': 'Wine', 'type': 'classification', 'table': df2},
        ...
    ]

    خروجی: DataFrame خلاصه از بیشترین اختلاف برای هر دیتاست
    """
    summary_rows = []

    for item in dataset_tables:
        name = item['name']
        dtype = item['type']
        df = item['table']

        if dtype == 'regression':
            # محاسبه بیشترین اختلاف MSE
            df['MSE_Diff'] = df['All_MSE'] - df['BestGroup_MSE']
            best_row = df.loc[df['MSE_Diff'].idxmax()]
            summary_rows.append({
                'Dataset': name,
                'Type': dtype,
                'All_Accuracy': None,
                'BestGroup_Accuracy': None,
                'All_R2': best_row['All_R2'],
                'BestGroup_R2': best_row['BestGroup_R2'],
                'All_MSE': best_row['All_MSE'],
                'BestGroup_MSE': best_row['BestGroup_MSE']
            })

        elif dtype == 'classification':
            # محاسبه بیشترین اختلاف دقت
            df['Accuracy_Diff'] = df['BestGroup_Accuracy'] - df['All_Accuracy']
            best_row = df.loc[df['Accuracy_Diff'].idxmax()]
            summary_rows.append({
                'Dataset': name,
                'Type': dtype,
                'All_Accuracy': best_row['All_Accuracy'],
                'BestGroup_Accuracy': best_row['BestGroup_Accuracy'],
                'All_R2': None,
                'BestGroup_R2': None,
                'All_MSE': None,
                'BestGroup_MSE': None
            })

    return pd.DataFrame(summary_rows)


In [ ]:
results = [
    {'name': 'Diabetes', 'type': 'regression', 'table': df_diabetes},
    {'name': 'BodyFat', 'type': 'regression', 'table': df_bodyfat},
    {'name': 'Wine', 'type': 'classification', 'table': df_wine},
    {'name': 'Cancer', 'type': 'classification', 'table': df_cancer},
]

summary = summarize_max_diff_results(results)
print(summary)
